In [1]:
import plotly.io as pio
pio.renderers.default = 'iframe'

In [2]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import yfinance as yf
import vectorbt as vbt
from datetime import datetime

import seaborn as sns
import mplfinance as mpf

In [6]:
symbols = ['BTC-USD', 'ETH-USD', 'XRP-USD'] #, 'BNB-USD', 'BCH-USD', 'LTC-USD']
start = datetime(2020, 1, 23)  # crypto is UTC
end = datetime(2020, 5, 1)

In [7]:
# OHLCV by column
ohlcv = vbt.YFData.download(symbols, start=start, end=end).concat()
# ohlcv = vbt.YFData.download(symbols).get('Close')
ohlcv
pd.concat(ohlcv, axis=1)

Open                               High  \
symbol                         BTC-USD     ETH-USD   XRP-USD      BTC-USD   
Date                                                                        
2020-01-22 00:00:00+00:00  8744.210938  169.608536  0.237263  8792.994141   
2020-01-23 00:00:00+00:00  8680.650391  168.295700  0.236173  8687.747070   
2020-01-24 00:00:00+00:00  8405.567383  162.899368  0.226201  8514.666992   
2020-01-25 00:00:00+00:00  8440.119141  163.067291  0.223049  8458.453125   
2020-01-26 00:00:00+00:00  8364.410156  161.176819  0.220345  8602.401367   
...                                ...         ...       ...          ...   
2020-04-26 00:00:00+00:00  7570.139160  195.413589  0.195033  7700.594238   
2020-04-27 00:00:00+00:00  7679.418945  197.475723  0.196700  7795.601074   
2020-04-28 00:00:00+00:00  7796.970215  197.273514  0.198046  7814.527344   
2020-04-29 00:00:00+00:00  7806.712402  198.465195  0.215562  8871.753906   
2020-04-30 00:00:00+00:00  8797.669922  216.909134  0.227169  9440.650391   

                                                         Low              \
symbol                        ETH-USD   XRP-USD      BTC-USD     ETH-USD   
Date                                                                       
2020-01-22 00:00:00+00:00  171.008698  0.238689  8636.747070  166.774231   
2020-01-23 00:00:00+00:00  168.312241  0.236284  8333.637695  160.295319   
2020-01-24 00:00:00+00:00  164.309448  0.226314  8266.840820  156.749741   
2020-01-25 00:00:00+00:00  163.227234  0.223150  8296.218750  158.632477   
2020-01-26 00:00:00+00:00  168.220322  0.231344  8325.498047  160.281128   
...                               ...       ...          ...         ...   
2020-04-26 00:00:00+00:00  199.344971  0.197215  7561.407715  194.768066   
2020-04-27 00:00:00+00:00  199.552795  0.199275  7679.418945  193.454163   
2020-04-28 00:00:00+00:00  198.786545  0.218108  7730.806641  194.849426   
2020-04-29 00:00:00+00:00  218.454636  0.229627  7786.049316  198.124512   
2020-04-30 00:00:00+00:00  227.529694  0.235703  8533.255859  206.436920   

                                           Close  ...                 Volume  \
symbol                      XRP-USD      BTC-USD  ...   XRP-USD      BTC-USD   
Date                                              ...                          
2020-01-22 00:00:00+00:00  0.234236  8680.875977  ...  0.236056  22600204051   
2020-01-23 00:00:00+00:00  0.223069  8406.515625  ...  0.226104  25770680779   
2020-01-24 00:00:00+00:00  0.215389  8445.434570  ...  0.223041  24397913026   
2020-01-25 00:00:00+00:00  0.217737  8367.847656  ...  0.220427  19647331549   
2020-01-26 00:00:00+00:00  0.219327  8596.830078  ...  0.230686  22177678796   
...                             ...          ...  ...       ...          ...   
2020-04-26 00:00:00+00:00  0.194504  7679.867188  ...  0.196805  33070154491   
2020-04-27 00:00:00+00:00  0.195437  7795.601074  ...  0.197978  36162144725   
2020-04-28 00:00:00+00:00  0.196372  7807.058594  ...  0.215484  33187959921   
2020-04-29 00:00:00+00:00  0.213436  8801.038086  ...  0.227432  60201052203   
2020-04-30 00:00:00+00:00  0.211577  8658.553711  ...  0.212761  66964629541   

                                                   Dividends                  \
symbol                         ETH-USD     XRP-USD   BTC-USD ETH-USD XRP-USD   
Date                                                                           
2020-01-22 00:00:00+00:00   9209827928  1517150861       0.0     0.0     0.0   
2020-01-23 00:00:00+00:00  10396868698  1600294983       0.0     0.0     0.0   
2020-01-24 00:00:00+00:00  10657671162  1706347379       0.0     0.0     0.0   
2020-01-25 00:00:00+00:00   8256956802  1373179560       0.0     0.0     0.0   
2020-01-26 00:00:00+00:00   9261861590  1583964764       0.0     0.0     0.0   
...                                ...         ...       ...     ...     ...   
2020-04-26 00:00:00+00:00  18335367012  169242752

In [15]:
def custom_ma(open, high, low, close, ma_period, symbols):
    # print(symbols)
    vbt_ma = vbt.IndicatorFactory.from_pandas_ta('sma')
    entries_result = {}
    exits_result = {}

    for i, symbol in enumerate(symbols):
        ma = vbt_ma.run([item[i] for item in close], length=ma_period)
        ma_df = pd.DataFrame(data=ma.sma.values, columns=['SMA_'+str(ma_period)], index=ma.sma.index)
        _df = pd.concat([ma_df], axis=1)

        _df['open'] = [item[i] for item in open]
        _df['high'] = [item[i] for item in high]
        _df['low'] = [item[i] for item in low]
        _df['close'] = [item[i] for item in close]

        _df['prev_close'] = _df['close'].shift(1)
        _df['prev_ma'] = _df['SMA_'+str(ma_period)].shift(1)
        _df['entries'] = (_df['close'] > _df['SMA_'+str(ma_period)]) & (_df['prev_close'] < _df['prev_ma'])
        _df['exits'] = (_df['close'] < _df['SMA_'+str(ma_period)]) & (_df['prev_close'] > _df['prev_ma'])

        entries_result[f'{symbol}'] = _df['entries']
        exits_result[f'{symbol}'] = _df['exits']

    entries_result = pd.concat(entries_result, axis=1)
    exits_result = pd.concat(exits_result, axis=1)
    return entries_result, exits_result

In [17]:
df = pd.concat(ohlcv, axis=1).copy()
symbols = df.loc[:, 'Open'].columns.values
entries, exits = custom_ma(df['Open'].values, df['High'].values, df['Low'].values, df['Close'].values,    
                    ma_period = 10, symbols=symbols
                    )

In [9]:
entries

port = vbt.Portfolio.from_signals(df.Close,
                                direction = 'long_only',
                                freq = '1d',
                                init_cash = 10000,
                                fees = 0.0017,
                                slippage = 0.000,
                                size_type = 'Percent',
                                size = 1,
                                entries = entries,
                                exits = exits
                                )

In [10]:
port.orders.records_readable

,Order Id,Column,Timestamp,Size,Price,Fees,Side
0,0,"(BTC-USD, BTC-USD)",2020-02-05 00:00:00+00:00,1.038447,9613.423828,16.971149,Buy
1,1,"(BTC-USD, BTC-USD)",2020-02-15 00:00:00+00:00,1.038447,9889.424805,17.458390,Sell
2,2,"(BTC-USD, BTC-USD)",2020-02-18 00:00:00+00:00,1.009149,10141.996094,17.399132,Buy
3,3,"(BTC-USD, BTC-USD)",2020-02-19 00:00:00+00:00,1.009149,9633.386719,16.526586,Sell
4,4,"(BTC-USD, BTC-USD)",2020-02-23 00:00:00+00:00,0.976221,9924.515625,16.470491,Buy
5,5,"(BTC-USD, BTC-USD)",2020-02-24 00:00:00+00:00,0.976221,9650.174805,16.015201,Sell
6,6,"(BTC-USD, BTC-USD)",2020-03-05 00:00:00+00:00,1.034142,9078.762695,15.960842,Buy
7,7,"(BTC-USD, BTC-USD)",2020-03-08 00:00:00+00:00,1.034142,8108.116211,14.254405,Sell
8,8,"(BTC-USD, BTC-USD)",2020-03-19 00:00:00+00:00,1.349737,6191.192871,14.206022,Buy
9,9,"(BTC-USD, BTC-USD)",2020-03-28 00:00:00+00:00,1.349737,6242.193848,14.323047,Sell


In [11]:
port.stats()

C:\Users\A715-72G\AppData\Local\Temp/ipykernel_27832/3103083911.py:1: UserWarning:

Object has multiple columns. Aggregating using <function mean at 0x00000230F7C1F310>. Pass column to select a single column/group.



Start                         2020-01-22 00:00:00+00:00
End                           2020-04-30 00:00:00+00:00
Period                                100 days 00:00:00
Start Value                                     10000.0
End Value                                  10252.723096
Total Return [%]                               2.527231
Benchmark Return [%]                           4.410365
Max Gross Exposure [%]                            100.0
Total Fees Paid                              185.785198
Max Drawdown [%]                              26.159584
Max Drawdown Duration                  61 days 08:00:00
Total Trades                                   6.666667
Total Closed Trades                            5.666667
Total Open Trades                                   1.0
Open Trade PnL                              1581.624377
Win Rate [%]                                  27.619048
Best Trade [%]                                14.245097
Worst Trade [%]                              -12

In [18]:
Ma_indy = vbt.IndicatorFactory(class_name='Ma_indy',
                                input_names=['open', 'high', 'low', 'close'],
                                param_names=['ma_period'],
                                output_names=['entries', 'exits'],
                               ).from_apply_func(custom_ma, ma_period = 10)

In [20]:
df_test = pd.concat(ohlcv, axis=1).copy()
symbols = df_test.loc[:, 'Open'].columns.values
result = Ma_indy.run(df_test.Open, df_test.High, df_test.Low, df_test.Close,
            ma_period=np.arange(50, 70, 10),
            symbols=symbols,
            param_product=True)

In [25]:
result.entries

pf = vbt.Portfolio.from_signals(df_test.Close,
                                direction = 'long_only',
                                freq = '1d',
                                init_cash = 10000,
                                fees = 0.0017,
                                slippage = 0.000,
                                size_type = 'Percent',
                                size = 1,
                                entries = result.entries,
                                exits = result.exits
                                )

In [26]:
pf.orders.records_readable

,Order Id,Column,Timestamp,Size,Price,Fees,Side
0,0,"(50, BTC-USD)",2020-04-16 00:00:00+00:00,1.402740,7116.804199,16.971149,Buy
1,1,"(50, ETH-USD)",2020-04-16 00:00:00+00:00,57.987807,172.157379,16.971149,Buy
2,2,"(50, XRP-USD)",2020-04-08 00:00:00+00:00,49699.202711,0.200869,16.971149,Buy
3,3,"(50, XRP-USD)",2020-04-09 00:00:00+00:00,49699.202711,0.198257,16.750465,Sell
4,4,"(50, XRP-USD)",2020-04-16 00:00:00+00:00,51654.731569,0.190104,16.693610,Buy
5,5,"(50, XRP-USD)",2020-04-20 00:00:00+00:00,51654.731569,0.183490,16.112815,Sell
6,6,"(50, XRP-USD)",2020-04-21 00:00:00+00:00,51245.362421,0.184328,16.058124,Buy
7,7,"(60, BTC-USD)",2020-04-23 00:00:00+00:00,1.343661,7429.724609,16.971149,Buy
8,8,"(60, ETH-USD)",2020-04-18 00:00:00+00:00,53.409743,186.914001,16.971149,Buy
9,9,"(60, ETH-USD)",2020-04-20 00:00:00+00:00,53.409743,172.297165,15.643991,Sell


In [38]:
comb_ret = pf.total_return()
pf.total_return()

ma_indy_ma_period  symbol 
50                 BTC-USD    0.214570
                   ETH-USD    0.203839
                   XRP-USD    0.090301
60                 BTC-USD    0.163416
                   ETH-USD    0.102216
                   XRP-USD    0.107917
Name: total_return, dtype: float64

In [35]:
comb_ret.loc[comb_ret.values==comb_ret.max()]

ma_indy_ma_period  symbol 
50                 BTC-USD    0.21457
Name: total_return, dtype: float64

In [40]:
for index, ret in comb_ret.loc[comb_ret.values==comb_ret.max()].iteritems():
    print(index, ret)

(50, 'BTC-USD') 0.21457031954465364


In [66]:
# df.loc[:, 'Open'].columns[1]
df['Open']['BTC-USD'].values

array([ 8744.2109375 ,  8680.65039062,  8405.56738281,  8440.11914062,
        8364.41015625,  8597.30859375,  8912.52441406,  9357.47070312,
        9316.01660156,  9508.31347656,  9346.35742188,  9389.8203125 ,
        9344.68359375,  9292.84179688,  9183.41601562,  9617.82128906,
        9726.00292969,  9793.0703125 ,  9863.89453125, 10115.55957031,
        9855.89160156, 10202.38769531, 10323.9609375 , 10211.55078125,
       10313.85644531,  9889.1796875 ,  9936.56054688,  9691.23046875,
       10143.79882812,  9629.32519531,  9611.78222656,  9687.70703125,
        9663.31835938,  9921.58300781,  9651.3125    ,  9338.29003906,
        8825.09375   ,  8788.72851562,  8671.21289062,  8599.75878906,
        8563.26464844,  8865.38769531,  8788.54199219,  8760.28515625,
        9078.30859375,  9121.60058594,  8908.20605469,  8111.14648438,
        7922.14697266,  7910.08984375,  7913.61621094,  5017.83105469,
        5573.07763672,  5201.06689453,  5385.22949219,  5002.578125  ,
      

In [11]:
coin = ["BTC-USD", "ETH-USD",'BNB-USD']
price = vbt.YFData.download(coin).get('Close')
df = price.copy()
df

c:\Users\A715-72G\anaconda3\lib\site-packages\vectorbt\data\base.py:527: UserWarning:

Symbols have mismatching index. Setting missing data points to NaN.



symbol,BTC-USD,ETH-USD,BNB-USD
Date,,,
2014-09-17 00:00:00+00:00,457.334015,NaN,NaN
2014-09-18 00:00:00+00:00,424.440002,NaN,NaN
2014-09-19 00:00:00+00:00,394.795990,NaN,NaN
2014-09-20 00:00:00+00:00,408.903992,NaN,NaN
2014-09-21 00:00:00+00:00,398.821014,NaN,NaN
...,...,...,...
2023-02-05 00:00:00+00:00,22955.666016,1631.645874,327.869354
2023-02-06 00:00:00+00:00,22760.109375,1616.247070,324.627167
2023-02-07 00:00:00+00:00,23264.291016,1672.003540,333.227875


In [15]:
df.columns.name='Ticker'
windows = np.arange(5, 50)
fast_ma, slow_ma = vbt.MA.run_combs(df, window=windows,ewm=True, r=2, short_names=['fast', 'slow']) 

entries = fast_ma.ma_crossed_above(slow_ma) 
exits = fast_ma.ma_crossed_below(slow_ma)

pf_kwargs = dict(size=np.inf, fees=0.0015, freq='1D') 
pf = vbt.Portfolio.from_signals(df, entries, exits,direction = 'both', **pf_kwargs)

comb_ret = pf.total_return()

In [18]:
entries

fast_window                    5                                           \
fast_ewm                     True                                           
slow_window                    6                       7                    
slow_ewm                     True                    True                   
Ticker                    BTC-USD ETH-USD BNB-USD BTC-USD ETH-USD BNB-USD   
Date                                                                        
2014-09-17 00:00:00+00:00   False   False   False   False   False   False   
2014-09-18 00:00:00+00:00   False   False   False   False   False   False   
2014-09-19 00:00:00+00:00   False   False   False   False   False   False   
2014-09-20 00:00:00+00:00   False   False   False   False   False   False   
2014-09-21 00:00:00+00:00   False   False   False   False   False   False   
...                           ...     ...     ...     ...     ...     ...   
2023-02-05 00:00:00+00:00   False   False   False   False   False   False   
2023-02-06 00:00:00+00:00   False   False   False   False   False   False   
2023-02-07 00:00:00+00:00   False   False   False   False   False   False   
2023-02-08 00:00:00+00:00   False   False   False   False   False   False   
2023-02-09 00:00:00+00:00   False   False   False   False   False   False   

fast_window                                                ...      46  \
fast_ewm                                                   ...    True   
slow_window                    8                       9   ...      49   
slow_ewm                     True                    True  ...    True   
Ticker                    BTC-USD ETH-USD BNB-USD BTC-USD  ... BNB-USD   
Date                                                       ...           
2014-09-17 00:00:00+00:00   False   False   False   False  ...   False   
2014-09-18 00:00:00+00:00   False   False   False   False  ...   False   
2014-09-19 00:00:00+00:00   False   False   False   False  ...   False   
2014-09-20 00:00:00+00:00   False   False   False   False  ...   False   
2014-09-21 00:00:00+00:00   False   False   False   False  ...   False   
...                           ...     ...     ...     ...  ...     ...   
2023-02-05 00:00:00+00:00   False   False   False   False  ...   False   
2023-02-06 00:00:00+00:00   False   False   False   False  ...   False   
2023-02-07 00:00:00+00:00   False   False   False    True  ...   False   
2023-02-08 00:00:00+00:00   False   False   False   False  ...   False   
2023-02-09 00:00:00+00:00   False   False   False   False  ...   False   

fast_window                    47                                          \
fast_ewm                     True                                           
slow_window                    48                      49                   
slow_ewm                     True                    True                   
Ticker                    BTC-USD ETH-USD BNB-USD BTC-USD ETH-USD BNB-USD   
Date                                                                        
2014-09-17 00:00:00+00:00   False   False   False   False   False   False   
2014-09-18 00:00:00+00:00   False   False   False   False   False   False   
2014-09-19 00:00:00+00:00   False   False   False   False   False   False   
2014-09-20 00:00:00+00:00   False   False   False   False   False   False   
2014-09-21 00:00:00+00:00   False   False   False   False   False   False   
...                           ...     ...     ...     ...     ...     ...   
2023-02-05 00:00:00+00:00   False   False   False   False   False   False   
2023-02-06 00:00:00+00:00   False   False   False   False   False   False   
2023-02-07 00:00:00+00:00   False   False   False   False   False   False   
2023-02-08 00:00:00+00:00   False   False   False   False   False   False   
2023-02-09 00:00:00+00:00   False   False   False   False   False   False   

fast_window                    48                  
fast_ewm                     True                  
slow_window      